In [1]:
import cv2
import time
import PIL
import numpy as np
from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
def normalize_images(ims):
    return ims / 255.0

def load_model(model_file='model.json', weights_file='model.h5'):
    json_file = open(model_file, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    
    # load weights into new model
    loaded_model.load_weights(weights_file)
    print("Loaded model from disk")
    return loaded_model

def get_gender(model, arr, shape=(64, 64)):
    im = PIL.Image.fromarray(arr)
    good_size_arr = np.array(im.resize(shape))
    if len(arr.shape) == 2:
        good_size_arr = np.array([good_size_arr, good_size_arr, good_size_arr])
    cls = model.predict(np.array([normalize_images(good_size_arr)]))
    if cls[0][0] < 0.5:
        return "Female"
    else:
        return "Male"

class CameraDisplay:    
    def __init__(self, onFrame):
        self.windowName = "myWindow"
        self.showWindow = True
        self.onFrame = onFrame
        self.createWindowAndLoop()
    
    def createWindowAndLoop(self):
        def onSliderChanged(state):
            self.showWindow = False
    
        def onMouse(event, x, y, flags, param):
            if event == cv2.EVENT_LBUTTONDOWN:
                self.showWindow = False
        
        try:
            cap = cv2.VideoCapture(0)
            cv2.namedWindow(self.windowName, cv2.WINDOW_FULLSCREEN)
            cv2.createTrackbar("Slide to stop", self.windowName, 0, 1, onSliderChanged)
            cv2.setMouseCallback(self.windowName, onMouse)

            while(self.showWindow):
                # Capture frame-by-frame
                ret, frame = cap.read()
                if (frame == None):
                    print("Error on capturing frame")
                    break

                # Display the resulting frame
                self.onFrame(frame)
                cv2.imshow(self.windowName, frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

                time.sleep(0.1)
        finally:
            # When everything done, release the capture
            cap.release()
            cv2.destroyAllWindows()

In [3]:
size=(128, 128)
model = load_model()

def onFrame(frame):
    gender = get_gender(model, frame, size)
    cv2.putText(frame, gender, (100,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255))

cam = CameraDisplay(onFrame)

Loaded model from disk


C:\Users\liviu\Anaconda3\envs\ztdlg\lib\site-packages\ipykernel_launcher.py:50: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
